In [ ]:
# -*- coding:utf-8 -*-
import imageio
import torch
from torchvision import transforms
from torchvision import models
import torch.nn as nn
from matplotlib import pyplot as plt
import pandas as pd
import os
from torch import optim



In [ ]:
print(torch.cuda.device_count())

In [ ]:
# Define a class to create dataset
class Data_curves(torch.utils.data.Dataset):
    def __init__(self, label_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(label_file)
        self.img_dir = img_dir
        self.transform = transforms.ToTensor()
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        #print(img_path)
        image = imageio.imread(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label
    

img1_dir = "trainFile_path"    
label1_file = "trainLabelFile_path"

img2_dir = "testFile_path"    
label2_file = "testFile_path"

train_curves = Data_curves(label1_file, img1_dir)
test_curves = Data_curves(label2_file, img2_dir)

In [ ]:
# Use utils.data.Dataloader importing dataset
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_curves)
test_dataloader = DataLoader(test_curves)

In [ ]:
# Select ResNet18 finishing pre-training process
resnet18 = models.resnet18(weights=None)
resnet18.fc = nn.Sequential(nn.Linear(in_features=512,out_features=5),
                            
                        nn.LogSoftmax(dim=1))

In [ ]:
# Pre-training process setting
import datetime  

def training_loop(n_epochs, optimizer, model, loss_fn, train_dataloader):
    for epoch in range(1, n_epochs + 1):  
        loss_train = 0.0
        for imgs, labels in train_dataloader:     
            
            labels = labels.to(device)
            
            imgs_t = imgs.to(device)
            
            #imgs_t = imgs_t.type(torch.float32)
            
            outputs = model(imgs_t)
            
            loss = loss_fn(outputs, labels)  

            optimizer.zero_grad()  
            
            loss.backward()  
            
            optimizer.step()  

            loss_train += loss.item()  

        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_dataloader)))  


In [ ]:
# Pre-training Process
train_loader = torch.utils.data.DataLoader(train_curves, batch_size=32,
                                           shuffle=True) 

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model = Net().to(device)
model = resnet34.to(device)

optimizer = optim.SGD(model.parameters(), lr=2e-3)
loss_fn = nn.CrossEntropyLoss()

training_loop( 
    n_epochs = 150,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_dataloader = train_loader,
)

In [ ]:
model = torch.save(model, "pre_train_model.pt")

In [ ]:
val_loader = torch.utils.data.DataLoader(test_curves, batch_size=1, 
                                          shuffle=False)

correct = 0
total = 0
model = torch.load("pre_train_model.pt")

with torch.no_grad():
    for imgs, labels in val_loader:
        labels = labels.to(device)
        imgs = imgs.to(device)
        outputs = model(imgs)
        _, predicted = torch.max(outputs, dim=1)
        print(f"predicted_result：{predicted}, label：{labels}")
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
print(f"accu：{correct/total}")